
## Create Table in Database

In [2]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

In [3]:


user='martin_svitek'
password='T67wni047Ugy2uUx'

In [4]:
# Conection
host = 'localhost'
port = '5433'
database = 'layereddb'
schema='berlin_source_data'

#connection to db after i opened tunnel
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [ ]:
#create table with constraints and references first
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.test_table_{user} (
    id INTEGER PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    score INTEGER
);
"""

# Execute the query to create my empty table
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction

In [ ]:
#  Send the DataFrame to the database using .to_sql()
df.to_sql(
    f'veterinary_clinics_{user_name}',        # table name
    engine,
    schema=schema,
    if_exists='append', # append table if it exists
    index=False
)

print("DataFrame sent to PostgreSQL using .to_sql() with psycopg2!")

In [5]:
##let's query test data!
query = f"""
SELECT * from berlin_source_data.districts
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df


,district_id,district,geometry
0,11012012,Reinickendorf,0106000020E61000000100000001030000000100000084...
1,11004004,Charlottenburg-Wilmersdorf,0106000020E6100000010000000103000000010000000D...
2,11009009,Treptow-Köpenick,0106000020E610000001000000010300000001000000E9...
3,11003003,Pankow,0106000020E61000000400000001030000000100000012...
4,11008008,Neukölln,0106000020E610000001000000010300000001000000BF...
5,11011011,Lichtenberg,0106000020E61000000100000001030000000100000038...
6,11010010,Marzahn-Hellersdorf,0106000020E6100000010000000103000000010000005A...
7,11005005,Spandau,0106000020E6100000010000000103000000010000001D...
8,11006006,Steglitz-Zehlendorf,0106000020E6100000010000000103000000010000008A...
9,11001001,Mitte,0106000020E610000001000000010300000001000000F3...
